<a href="https://colab.research.google.com/github/Maviya5/CE888/blob/master/Lab5/Different_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
user_ratings_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")
user_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_features.csv")
item_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/item_features.csv")

In [0]:
user_features_df["key"] = 0
user_features_df["user_id"] = range(user_features_df.shape[0])
item_features_df["key"] = 0
item_features_df["item_id"] = range(item_features_df.shape[0])

merged_df = pd.merge(user_features_df, item_features_df, left_index=True, on="key")
merged_df.drop(labels='key', axis=1, inplace=True)  # drop the "key" column

In [4]:
merged_df[["item_id", "user_id"]]


,item_id,user_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [0]:
merged_df["rating"] = map(lambda ids: user_ratings_df.values[ids[1]][ids[2]], 
                          merged_df[["user_id", "item_id"]].itertuples())

train = merged_df.dropna()
test = merged_df[merged_df.isnull().any(axis=1)]

In [6]:
n_latent_factors = 2

user_ratings = user_ratings_df.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

user_features = user_features_df.values
item_features = item_features_df.values

user_features = np.concatenate([np.ones(shape=(user_features.shape[0],1)), user_features], axis=1)
item_features = np.concatenate([np.ones(shape=(item_features.shape[0],1)), item_features], axis=1)

user_features_weights = np.random.random((user_ratings.shape[0], user_features.shape[1]))
item_features_weights = np.random.random((user_ratings.shape[1], item_features.shape[1]))

print(user_features)

[[1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 2.]
 [1. 1. 0. 0. 3.]
 [1. 0. 1. 0. 4.]
 [1. 0. 0. 0. 5.]
 [1. 0. 0. 0. 6.]
 [1. 1. 0. 0. 7.]
 [1. 0. 1. 0. 8.]
 [1. 1. 0. 0. 9.]]


In [0]:
def predict_rating(user_id, item_id):
    """
    Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    
    user_score = user_features_weights[user_id].dot(user_features[user_id])
    item_score = item_features_weights[item_id].dot(item_features[item_id])
    #print(user_preference.dot(item_preference), user_score, item_score)
    return user_preference.dot(item_preference) + user_score + item_score


def train(user_id, item_id, rating, alpha=0.001, 
          latent_feature_weight_decay=0.1,
          user_weight_decay=0.01, item_weight_decay=0.0001):
    #print(item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err = prediction_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id] + latent_feature_weight_decay * latent_user_preferences[user_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values + latent_feature_weight_decay * latent_item_features[item_id])
    
    user_features_weights[user_id] -=alpha * err * (user_features[user_id] + user_weight_decay * user_features_weights[user_id])
    item_features_weights[item_id] -=alpha * err * (item_features_weights[item_id] + item_weight_decay * item_features_weights[item_id])
    
    return err


def sgd(iterations=30000):
    """ 
    Iterate over all users and all items and train for 
    a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print(mse)

In [8]:

for _ in range(10): 
    sgd()  # Note decreasing values with increasing iterations

0.2783338355111144


KeyboardInterrupt: ignored

In [9]:
predictions = np.zeros(shape=(latent_user_preferences.shape[0], latent_item_features.shape[0]))
print(user_features_weights)
print(item_features_weights)
for user_id in range(latent_user_preferences.shape[0]):
    for item_id in range(latent_item_features.shape[0]):
        predictions[user_id, item_id] =  predict_rating(user_id, item_id)

[[ 0.929  0.74   0.626  0.256  0.442]
 [-0.679  0.826 -0.619  0.675 -0.67 ]
 [ 0.382  0.294  0.298  0.299  0.951]
 [ 0.786  0.676  0.043  0.28   0.691]
 [-0.084  0.392  0.317  0.782  0.063]
 [ 0.696  0.39   0.288  0.635  1.665]
 [ 0.094  0.574  0.544  0.842  0.014]
 [ 0.781  0.671  0.229  0.387  0.03 ]
 [ 0.476  0.783  0.364  0.158 -0.503]
 [ 0.159  0.445  0.769  0.19  -0.226]]
[[1.609 0.445 1.569 0.197 0.753]
 [0.079 0.007 0.125 0.08  0.143]
 [0.044 0.265 0.048 0.3   0.41 ]
 [0.53  0.071 0.793 0.656 0.677]
 [1.684 0.462 1.885 0.464 1.543]]


In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = user_ratings_df.columns

In [11]:
comparison_data


,The Call of Cthulhu,Frankenstein,Dracula,Neuromancer,Space Odyssey
0,"(8.0, 7.942667299461632)","(2.0, 2.1957018223404035)","(nan, 16.389696868819104)","(5.0, 4.886453987105739)","(4.0, 3.9752907697882858)"
1,"(3.0, 2.924449001593395)","(2.0, 2.34172982329126)","(nan, -21.44740866600967)","(7.0, 6.781907748789935)","(7.0, 6.959615184454863)"
2,"(9.0, 8.981720257224126)","(nan, 4.489220192053177)","(7.0, 7.003146496403881)","(8.0, 8.014422644755289)","(5.0, 4.998547483187301)"
3,"(nan, 8.822326839067358)","(nan, 4.938927646109542)","(7.0, 6.999883294002647)","(8.0, 7.998348144280428)","(9.0, 8.999615520266365)"
4,"(nan, 4.93052596390614)","(1.0, 0.7461916573791169)","(8.0, 8.008494465603661)","(3.0, 3.2049647922417415)","(7.0, 7.02815449496973)"
5,"(2.0, 2.0260710454808946)","(3.0, 2.973640325905375)","(5.0, 4.997714868708794)","(nan, 1.7453265828927829)","(nan, 38.04148617242404)"
6,"(4.0, 4.433810436792561)","(2.0, 0.4437975500608944)","(nan, 6.721011854510673)","(2.0, 2.8942681083984616)","(7.0, 7.187329907483002)"
7,"(7.0, 6.2583335701428995)","(1.0, 3.008234653851697)","(2.0, 2.0880215148632875)","(7.0, 6.002330830597849)","(9.0, 8.75632491634645)"
8,"(3.0, 3.2086486368183333)","(3.0, 2.293782805006235)","(nan, -23.618406118423604)","(7.0, 7.38352714478358)","(3.0, 3.081976699664981)"
9,"(4.0, 4.223378615272756)","(nan, -0.18127868251614632)","(5.0, 4.980258906503106)","(3.0, 2.823949606339715)","(3.0, 3.0072057745456684)"
